# Descenso del gradiente
>Es un algoritmo **iterativo** de **optimización** para encontrar el **mínimo** local de una función.

Cuando un modelo de machine learning aprende, no es que mejore directamente, en realidad va siendo "menos malo".   
Para lograr esto definimos una **loss function** también llamada *cost function* o **función de costo** que mide el **error** de nuestro modelo y encontramos el punto donde este sea **mínimo**, mediante la **gradiente** -de ahí el nombre- un vector de derivadas que apunta al **máximo** de dicha función con una pendiente más o menos inclinada dependiendo al error en ese punto.

<img src="https://cdn-images-1.medium.com/max/1000/1*RxU9mwBejyPoxM95_p-gEA.gif" width=400/>

Llegar al mínimo ¿apuntando al máximo?  
Como la idea es ser menos malo, nos **alejamos** del punto **máximo** de la función de costo, donde el error es **alto**, caminamos sobre ella dando pasos hacia **atrás** hasta alcanzar el punto **mínimo**.

Estos puntos de la función están determinados por los parámetros-$m$ y $b$ para la recta- y los datos de entrenamiento, pues comparamos el **modelo** -dado por los datos y los parametros- con los **targets**.

Por cada paso un modelo de regresión lineal cambiará más o menos de esta forma:

<img src="https://miro.medium.com/max/1280/1*eeIvlwkMNG1wSmj3FR6M2g.gif" width=400/>

## Función de costo
Esta cambia según el modelo que usemos, para las regresiones lineales se usa la diferencia entre predicción y target elevada al cuadrado, la misma idea de la métrica MSE.

Podemos ver, en la primera imagen de este capítulo, que tiene la forma $y = x^2$ que es [convexa](https://es.wikipedia.org/wiki/Funci%C3%B3n_convexa).  
Esto significa que tendrá un mínimo **global** pues no habrá otro mínimo, que denominaríamos local, como podría suceder con otra función más compleja:

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Extrema_example_es.svg/1200px-Extrema_example_es.svg.png" width=350/>

La ubicación deñ punto **mínimo** de esta función será **dependiente de los datos** de entrenamiento, que pueden ser muy diversos, y cambia de lugar si realizamos transformaciones o añadimos features. Es estas variaciones que de antemano no sabemos dónde está exactamente, ni por dónde empezar a buscarlo.

El algoritmo del descenso por el gradiente propone empezar con un punto **cualquiera**, iniciando los valores de los parámetros al azar, medir el error y **actualizarlos** para disminuir ese error, **iterando** este procedimiento, esto es, **repetirlo varias veces** hasta que nuestro punto tenga el error **mínimo**, momento en que alcanza la **convergencia**. 😃

Podemos controlar qué tanto cambiarán los parámetros con el hiperparámetro **learning rate** o ratio de aprendizaje.

## Learning rate
Podríamos decir que controla el tamaño de los pasos para actualizar los parámetros, pero debemos tener cuidado al definir este hiperparámetro.

Si el **learning rate** fuera **alto**, el modelo bien podría converger en **menos iteraciones**, pero si fuese **demasiado alto** daríamos pasos tan grandes que nos pasaríamos del punto mínimo, saltando la convergencia ☹.

Pero si fuera **bajo**, el modelo tardaría más en converger, aunque con el suficiente tiempo llegaríamos al punto mínimo. Lento pero seguro 🐢.

<img src="https://www.math.purdue.edu/~nwinovic/figures/learning_rates.png" width=500/>

## Feature scaling
Si tuvieramos dos features nuestra función de costo sería en 3D, nuestro objetivo de llegar al punto mínimo se mantiene y se vería más o menos así:  

<img src="https://suniljangirblog.files.wordpress.com/2018/12/1-1.gif" width=350>

Pero estas features pueden tener rangos muy **diferentes**, por ejemplo, en nuestro dataset RM tiene un rango diferente a LSTAT 

In [ ]:
from sklearn.datasets import load_boston

X, y = load_boston(True)

print(f"Rango RM    {min(X[:, 5])} -  {max(X[:, 5])}")
print(f"Rango LSTAT {min(X[:, 12])} -  {max(X[:, 12])}")

Esto causa que nuestra función de costo quede así, vista desde arriba:

![features_unscaled](img/5.1.4_features_unscaled.png)

Si iniciamos el descenso por uno de los extremos más alejados de esta elipse, nos tomará más tiempo llegar al mínimo global que tenemos en el centro, la solución es **escalar** las features para que tengamos un círculo con las features en un rango de (-1, 1) o (0, 1).

Escalar al primer rango se conoce como **standarization**, y al segundo como **normalization**. Podemos aplicar ambos con sklearn, generalmente **no** se escala la variable target, el objetivo a predecir.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

np.random.seed(42)

X_selected = X[:, [5, 7]]
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.4)

norm = MinMaxScaler()
stand = StandardScaler()
X_train_norm = norm.fit_transform(X_train)
X_train_stand = stand.fit_transform(X_train)

print("\t Sin escalar \t|\t Normalization \t|\t Standarization")
print( np.column_stack((X_train, X_train_norm, X_train_stand))[:5])

Podemos ver que standarization tiene negativos por su rango (-1, 1) también tiene la característica de llevar las features a una media de 0, es recomendable usarla cuando nuestros datos siguen una [distribución normal](https://es.wikipedia.org/wiki/Distribuci%C3%B3n_normal), en caso contrario, es recomendable usar normalization.

Pero son recomendaciones, es más importante probar y evaluar lo que mejor se ajuste a nuestros datos 😉

# Implementación
Vimos bastantes conceptos, ¿verdad? fue porque la idea principal es importante.

El algoritmo de este capítulo permite que machine learning sea machine learning, porque define una serie de pasos para que la máquina aprenda de los datos.
Para implementarlo debemos definir algunas fórmulas, específicas a la regresión lineal.

## Función de costo
$$ J(\theta) = \frac{1}{2m} \sum_{i=1}^{m} ( h_\theta(x^{(i)}) - y^{(i)} )^2$$

Donde:
+ $h_\theta(x) = \theta^T X = \theta_0 + \theta_1x_1 + \cdots + \theta_nx_n$
+ $m$ es la cantidad de datos
+ $i$ es el número de fila
+ $\theta$ es el vector de parámetros

Es muy parecida a MSE, con la diferencia de que multiplicamos todo por la constante $\tfrac{1}{2m}$  
$h_\theta(x)$ representa al modelo de regresión lineal múltiple, no es necesario incluir $x_0$ porque es igual a 1.  
Y almacenamos los parámetros en el vector $\theta$

### Operaciones vectorizadas
Para la computadora, suele ser más eficiente tratar directamente con vectores y matrices, que implementaremos usando **arrays de numpy**, muchas formulas usan una **notación vectorizada** como $\theta^T X$ para las **sumatorias de productos** aprovechando [cómo se calculan](https://www.problemasyecuaciones.com/matrices/multiplicar-matrices-producto-matricial-ejemplos-explicados-propiedades-matriz.html) las **multiplicaciones de matrices**, cuyo requisito es que el número de **columnas** de la primera sea **igual** al número de **filas** de la segunda, es decir sus **dimensiones** deben ser **(n, k)** y **(k, m)** respectivamente. El resultado será una matriz con dimensiones **(n, m)**. 

Esto significa que las matrices a multiplicar deben **compartir un mismo número**, en nuestro caso multiplicando `theta` por `X` ¿qué numero comparten?

`theta` es el vector de features y `X` es la matriz de datos de entrada, cada fila es un dato, y las columnas son... ¡features! ese es el número que comparten 😉

Para multiplicar usaremos el operador `@` porque así lo definió numpy, pero debemos ponerlos en el **orden correcto** la operación `theta @ X` multiplicaría arrays de `shape` **(f, 1)** y **(m, f)**, así no cumplimos el requisito.  
La fórmula multiplica la traspuesta de theta, implementada como `theta.T`, esto intercambia las dimensiones a **(1, f)** pero tampoco cumplimos el requsito.

La solución es multiplicar **(m, f)** x **(f, 1)** dando como resultado una matriz de **(m, 1)** igual que y, esto nos permitirá restar ambos elemento por elemento.

In [ ]:
def loss(X, y, theta):
    # el orden correcto (m, f) x (f, 1)
    # resultará en (m, 1) igual que y
    h = X @ theta
    
    # sólo transcribimos la fórmula, devolverá un número
    return 1 / (2*m) * np.sum((h - y) ** 2)

## Gradiente
Este es el vector de derivadas de la función de costo:

$$\frac{1}{m}\sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)})x_j^{(i)}$$

Donde $x_j^{(i)}$ es una feature en la fila número $i$ columna número $j$

Recuerda que la función de costo tiene la forma $x^2$ cuya derivada es $2x$ esto multiplicado por $\frac{1}{2m}$ resulta en $\frac{1}{m}$

Por su definición, debemos devolver un vector de shape **(f, 1)** para luego actualizar `theta` usando el **learning rate**, recuerda que tenemos 2 features y el término independiente, f = 3 en este caso.

Como en la función de costo, la operación `h - y` tendrá shape de **(m, 1)**, pero no la elevamos al cuadrado, debemos multiplicarla por X que tiene shape **(m, f)**.

Para esto ponemos trasponer `h - y` multiplicando **(1, m) x (m, f)**  
o `X` para multiplicar **(f, m) x (m, 1)**, tomaremos esta opción que resulta en **(f, 1)** como necesitamos 😉

In [ ]:
def grad(X, y, theta):
    h = X @ theta
    # cambiamos de lugar los términos
    return 1/m * X.T @ (h - y)

## Actualización
Por ultimo es importante utilizar lo que se conoce como la regla de actualización:

$$\theta_j := \theta_j - \alpha * grad \text{ (actualizar simultáneamente $\theta_j$ para todas las  $j$)}$$

Donde $\alpha$ es el **learning rate** y $j$ es el número de feature

Simultáneamente significa **al mismo tiempo** esta nota es importante porque **no debemos volver calcular el gradiente** hasta haber **actualizado todos** los parámetros.

## \# Código final
Seguro notaste que la implementación es un poco diferente a las fórmulas, esto pasa principalmente por las shapes de nuestras variables, y también porque resumimos la sumatoria de productos en forma la multiplicación de matrices para el caso de la gradiente 😃   
Por esto es **importante fijarse en las shapes**, en las que tenemos y en las que deseamos lograr, como la gradiente.

Primero definamos algunos hiperparámetros como **alpha** o el **número máximo de iteraciones**.  
Usaremos los datos de entrenamiento a los que aplicamos normalization, nos aseguramos que `y` sea una matriz de (m, 1) y a `X` le añadiremos una columna de unos, estos representan a $x_0$ para multiplicarlos con theta, que tendrá 3 elementos iniciados al azar antes de iterar.

In [ ]:
# aseguramos lareproducibilidad:
# mismos resultados en cada iteración
np.random.seed(42)

# Este suele ser un valor bajito
alpha = 0.001

# Tendremos no más de 1000 iteraciones
max_iters = 1000

# El viejo reshape
y_train = y_train.reshape(-1, 1)

# número de elementos
m = len(y_train)

# añadimos la columna de unos
X_train_norm_1 = np.column_stack((np.ones(m), X_train_norm))

# empezamos con un vector de números aleatorios
# tendrá 3 filas y 1 columna
theta = np.random.randn(3, 1)


for i in range(max_iters):
    # veamos cómo disminuye el costo cada 100 iteraciones
    if i % 100 == 0:
        costo = loss(X_train_norm_1, y_train, theta)
        print(f"Costo en it. # {i} {costo :.2f}")
        
    # primero calculamos el gradiente
    # luego actualizamos theta
    gradiente = grad(X_train_norm_1, y_train, theta)
    theta -= alpha * gradiente

El costo ha bajado, ¡el modelo ha aprendido! 😃

Prueba cambiando el **learning rate**, puedes ir en una escala de tres: 0.001, 0.003, 0.01, ...  
Mientras sea mayor, necesitarás menos **iteraciones** 😉

# La importancia de (transformar) los datos
El descenso del gradiente no es lo más importante para que las máquinas aprendan, son los **datos** en sí, porque se aprende de ellos y el como los transformamos **antes** de entrenar un modelo cambiará mucho los resultados que obtengamos, como en la **regresión polinómica** obtuvimos una **curva** sólo transformando los datos.

Esto se conoce como [preprocesamiento](6_preprocesamiento.ipynb).